# SELECT who FROM dunnit;

This notebook demonstrates how to create and query a local SQLite database using the `sqlalchemy` library. The database in question is based on [Cluedo](https://en.wikipedia.org/wiki/Cluedo), partly because the game involves a relatively simple data model, but mostly because the idea amused me.

The notebook will explore how to create, read, update, and delete data from the database: the four standard operations required for working with databases.

This notebook assumes that you are familiar with basic Python and SQL - 

Everything done in this notebook using `sqlalchemy` could have been done with the `sqlite3` library instead, with minor changes; this notebook uses `sqlalchemy`, but that's not at all the only valid choice.

## The situation

A man is dead.

Dr. Black was murdered in his own home. There are only six possible suspects, each with their own motivations and secrets. Someone, somewhere, with some weapon, killed the good doctor; that someone must be unmasked. There's no time to waste - the murderer must be brought to justice before they manage to erase any evidence of their crime, or even (god forbid) strike again. 

If you were a detective, then you'd be able to solve this crime for us with a combination of science, questioning, and logic. But you aren't. We're going to have to find the truth through a (luckily) much easier skill: structured query language (SQL).

In order to identify the killer, we need to design a database, collate all known data, and identify the murderer, the murder location, and the murder weapon. According to the rules of Cluedo, that information - and only that information - is required to make an accusation hold up in court.

## Imports

In [1]:
from sqlalchemy import create_engine  # Access database
import pandas as pd  # Dataframes and related methods
from itertools import product  # Generate all possible combinations

## Database construction

### Creating an engine

In order to access the database, we need to create an "engine" using `sqlalchemy`. This will manage our connections and queries when interacting with the database.

`create_engine()` requires one argument - a string that contains the type and name of the databse. As we're using SQLite to record data about Cluedo, the type is "sqlite" and the name is "cluedo.db". The tricky bit here is the punctuation between the type and the name. We need one colon - ":" - and **exactly three** forward slashes ("/"). With two or fewer slashes, it just won't work. With four slashes, it will look for a remote database, not a locally-hosted one.

One piece of functionality in SQLite that should be noted is that - if your database file does not exist - it will be created for you the first time you try and use it. This occasionally causes problems, as a slight error in the database name will mean that you're actually working with a different, newly-created database, rather than the one you intended.

In [2]:
engine = create_engine("sqlite:///cluedo.db")

### Creating tables

Once we've got the engine, we can start building the database tables inside it. In order to model the game of Cluedo, we'll use four tables - one each for **suspects**, **locations**, **weapons** and - linking all the other tables together - **accusations**.

Each of these tables will be created through an SQL query.

The engine **manages** connections, but it's not a direct connection itself: we don't use the engine directly to talk to the database. Instead, we open and close connections to the database using the engine. So far, the most helpful analogy I have found for this is that the engine is a telephone line connecting us to the database. Whenever we open a connection, we're picking up the phone. The telephone line is required to connect, but it isn't the active connection itself.

In [3]:
# Open a connection

conn = engine.connect()

With an open connection, we can create the first table. The SQL statement to do this is shown below.

In [4]:
statement = """
    CREATE TABLE IF NOT EXISTS suspect (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    honorific TEXT NOT NULL,
    name TEXT NOT NULL);
"""

The statement is stored as a variable - `statement` - so that we can refer to it with a shorter name later on, but actually create it inside a **multiline string**. The three speechmarks at the start and end of the string mean that we can include line breaks easily. This makes it a lot easier to write and read SQL statements without getting confused. Remember, SQL doesn't care about spaces or line breaks - you can arrange it in the way that makes more sense to you.

The statement itself `CREATE`s a table called `suspect`, but **only if that table does not exist already**. This means that you can re-run this cell later with an existing databse, and not hit an ugly error.

The `suspect` table contains three columns, one of which - `id` - is an **autoincrementing** primary key: each new row of data gets the next number in the sequence, uniquely identifying that row compared to all the others.

In [5]:
# Actually create the table

conn.execute(statement)

In [6]:
# Close the connection.

conn.close()

It's good practice to close your connection after each query (or closely-related set of queries in rapid succession), as this frees up resources and simplifies database management.

### Using "with"

Instead of explicitly opening and closing the connection each time, we can use the `with` keyword to automatically manage our connections.

The first step is to craft the next statement, which will create the `weapon` table. It works almost identically to the statement for the `suspect` table.

In [7]:
statement = """
    CREATE TABLE IF NOT EXISTS weapon (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL);
"""

The `with` keyword is a **context manager** - it helps deal with resources outside our program, such as the connections to a database. When we use `with`, we specify a resource to deal with, and a name for that resource. `with` will open the resource, and keep it open and accessible until the end of the `with` block. Once we've moved on from the `with`, the connection gets automatically closed.

This gives us a quick, two-line way to send statements to a database, without having to worry about connections.

In [8]:
# Using the context manager

with engine.connect() as conn:
    conn.execute(statement)

### Running multiple queries

SQLite is difficult about multiple statements; the easiest way to run multiple statements is to run each one separately in quick succession. This is we will create the last two tables in the database.

In [9]:
# Create the statements as a list

statements = [
    """
    CREATE TABLE IF NOT EXISTS location (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL);
    """,
    """
    CREATE TABLE IF NOT EXISTS accusation (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    suspect_id INTEGER NOT NULL,
    weapon_id INTEGER NOT NULL,
    location_id INTEGER NOT NULL,
    true INT,
    FOREIGN KEY (suspect_id) REFERENCES suspect(id),
    FOREIGN KEY (weapon_id) REFERENCES weapon(id),
    FOREIGN KEY (location_id) REFERENCES location(id));
    """
]

The query to create the `location` table is the same as we have seen before, but the one for `accusation` is a little more complicated. `accusation` is a table with thre foreign keys: one each for the **suspect**, **weapon**, and **location**. Each of these columns is declared as normal, but the last three lines of the statement forge the actual connections between the foreign key columns and their respective tables. Each foreign key column `REFERENCES` a column in another table.

There is also an integer `true` column: when justice has been served, this can be used to identify the correct accusation, with `0` representing a `True` accusation, and `1` a `False` one.

Once the statements have been written, we can use the `with` keyword and a quick loop to run each query, one after the other.

In [10]:
# Using the context manager

with engine.connect() as conn:
    
    # Loop through the statements
    
    for statement in statements:
        conn.execute(statement)

### Checking the table creation

Now that we've run all the statements, we should be able to see that the tables have been created. The `.table_names()` method of `engine` will list the existing tables.

In [11]:
# Show the tables

engine.table_names()

['accusation', 'location', 'sqlite_sequence', 'suspect', 'weapon']

The `sqlite_sequence` table is automatically created because we've used several `AUTOINCREMENT` columns in the database so far. The table keeps track of the largest integer assigned to a row in each table, so that new data can be given the next numbers available, and there is no possibility of collision.

## Inserting data

Now that we have tables, we can start adding data to them. Bear in mind that running these next cells multiple times will add multiple rows of data with identical information (except for primary key values). Re-running this notebook with an existing database will double the amount of data in it. If you end up with duplicate data you don't want, the quickest way to deal with it is to delete the `.db` file and then re-run this notebook: the correct tables and data will be recreated.

The simplest way to add data is with a single `INSERT` statement. We'll begin with Dr. Black, former owner of Tudor Close and current corpse.

### Inserting a single row

In [12]:
# Craft the statement

statement = """
    INSERT INTO suspect
    (honorific, name)
    VALUES
    ('Dr.', 'Black');
"""

There's no need to provide an `id` value, because the `AUTOINCREMENT` sorts that out for us, giving each row a unique identifier.

In [13]:
# Run the statement on the database

with engine.connect() as conn:
    conn.execute(statement)

### Inserting multiple rows

Of course, inserting data one line at a time is tedious, so we can instead insert multiple rows at once. Details for each row are contained in brackets, separated from each other by comments.

For the purposes of this notebook, we're only including characters from the original version of Cluedo, though you are - of course - free to add the rest of the characters if you feel strongly about it.

In [14]:
# Craft the statement

statement = """
    INSERT INTO suspect
    (honorific, name)
    VALUES
    ('Miss', 'Scarlett'),
    ('Colonel', 'Mustard'),
    ('Mrs.', 'White'),
    ('Reverend', 'Green'),
    ('Mrs', 'peacock'),
    ('Professor', 'Plum'),
    ('Dr.', 'Black');
"""

In [15]:
# Run the statement on the database - you may have noticed by now how reusable this snippet is

with engine.connect() as conn:
    conn.execute(statement)

## Inserting data from files

You might even want to load data from an existing file, to save you having to manually create a statement for hundreds, if not thousands, of lines. This is not actually a concern for this notebook, as the `location` table - which has the most rows - will still only contain details on nine separate places. However, to illustrate the principle, we'll load in the `weapon` and `location` data from files.

### Inserting file data using raw Python

You are unlikely to have to load data from a file and prepare SQL queries using just Python very often, but it's a very useful thing to be familiar with, even if you never use it. Thinking about data and SQL in these terms - as just text in a particular format - helps demystify various ideas around data, and make it easier to see what more advanced methods are actually doing.

The data on weapons is stored in a `.txt` file - `weapons.txt`. Using the `with` keyword, we can open and read data from this resource. We can read each line of the file and store the weapon information in a `list`.

In [16]:
# Create a holder list for the data

weapons = []

# Open the file with a context manager in "read" mode

with open("weapons.txt", "r") as file:
    
    # Loop through the file, line-by-line
    
    for line in file.readlines():
        
        # Add each line (stripped of extra spaces) to the holder
        
        weapons.append(line.strip())
        
# Display the weapon data

print(weapons)

['Candlestick', 'Dagger', 'Lead Pipe', 'Revolver', 'Rope', 'Wrench']


Once we've got the data into the program, we can craft an SQL statement using it. We build the basic stub of the statement, and then loop through each value in `weapons`, adding the weapon data (and appropriate punctuation) onto the end of the statement.

The appropriate punctuation is brackets and single quote marks around each value, a comma, and a newline character, just to make the formatting neater. In order to create that neatly, we're using [f-strings](https://realpython.com/python-f-strings/), a feature that lets us add data into strings by including it in curly brackets = "{}" - and putting an "f" character before the starting speechmark.

In [17]:
# Craft the statement

statement = """
INSERT INTO weapon
(name)
VALUES
"""

# Loop through the weapons

for weapon in weapons:
    
    # Add onto the statement
    
    statement = statement + f"('{weapon}'),\n"
    
# Remove the final character from the statement (trailing comma) and replace it with a semicolon

statement = statement[:-2] + ";"
    
# Display the statement

print(statement)


INSERT INTO weapon
(name)
VALUES
('Candlestick'),
('Dagger'),
('Lead Pipe'),
('Revolver'),
('Rope'),
('Wrench');


In [18]:
# Run the statement

with engine.connect() as conn:
    conn.execute(statement)

### Inserting file data using Pandas

As already mentioned, you won't (or will very rarely) need to craft your own queries using loops, operating on raw text data. We can rely on libraries like `Pandas` to create valid SQL for us automatically.

The data for the `location` table is stored in a `.csv` file: `locations.csv`. Using `Pandas`, we can load the data into a dataframe.

In [19]:
# Read the data into a dataframe

location_data = pd.read_csv("locations.csv")

With the data in a dataframe, it can be processed in any way required, and then sent straight to the database using the `.to_sql()` method.

The `.to_sql()` method takes a table name, and a connection object, and then adds all the rows of the dataframe into the database. In our case, we need to add two more arguments:

- `index=False` to stop the numeric index in the dataframe being passed in, because the `id` column in the `location` table handles the primary key for us
- `if_exists="append"` so that the rows are added onto the table, rather than overwriting it

In [20]:
# Open a connection (that will close itself later)

with engine.connect() as conn:
    
    # Add the data to the the dataframe
    
    location_data.to_sql("location", conn, index=False, if_exists="append")

## Reading data

Now that the tables have been created & filled, we can start reading data from the database. This has many familiar parts to methods previously used: we still need to open a connection and pass in some SQL, but now we get information coming back down the connection as well.

The response to a query is a `sqlalchemy` `ResultProxy`; in order to get information out of it, we need to call the `.fetchall()` method; this produces a `list` of `tuples`, one for each returned row.

In [21]:
# Open a connection

with engine.connect() as conn:
    
    # Run the query
    
    results = conn.execute("SELECT * FROM weapon;")
    
    # Extract the results from the results object
    
    results = results.fetchall()

In [22]:
# Display the results

print(results)

[(1, 'Candlestick'), (2, 'Dagger'), (3, 'Lead Pipe'), (4, 'Revolver'), (5, 'Rope'), (6, 'Wrench')]


### Reading data with Pandas

You don't have to read data in this format; you can also save time by reading data directly into `Pandas` using the `.read_sql()` method. As always, this requires a connection object.

In [23]:
# Craft the query - this one gets the suspects in alphabetical order

query = """
    SELECT *
    FROM suspect
    ORDER BY name ASC;
"""

In [24]:
# Open the connection using "with"

with engine.connect() as conn:
    suspects = pd.read_sql(query, conn, index_col="id")

Specifying `index_col` allows us to use the primary key of the table as our row index.

In [25]:
# Display the dataframe

suspects.head(9)

,honorific,name
id,,
1,Dr.,Black
8,Dr.,Black
5,Reverend,Green
3,Colonel,Mustard
7,Professor,Plum
2,Miss,Scarlett
4,Mrs.,White
6,Mrs,peacock


## Deleting data

We've made a terrible mistake: there are two Dr. Blacks in the database. Given the active murder investigation, having a [Doppelganger](https://en.wikipedia.org/wiki/Doppelg%C3%A4nger) of the murder victim wandering around causes unnecessary complications.

Luckily, deleting our duplicate doctor is no more difficult than inserting him was. Let's remove the one with the higher `id`.

In [26]:
# Craft the statement

statement = """
    DELETE FROM suspect
    WHERE id = 8
    AND honorific = 'Dr.';
"""

In [27]:
# Run the query

with engine.connect() as conn:
    conn.execute(statement)

## Updating data

We've made a second, though less problematic, mistake: Mrs. Peacock's name lacks the required capital letter.

Updating is just as easy as deleting.

In [28]:
# Craft the statement

statement = """
    UPDATE suspect
    SET name = 'Peacock'
    WHERE name = 'peacock';
"""

In [29]:
# Run the query

with engine.connect() as conn:
    conn.execute(statement)

## Reviewing the database

Our database should now be complete, correct, and populated. It's a good idea to just look at all the data, getting a good understanding of the current situation before working with it further.

The next cell combines several concepts that we've covered so far in order to print out each row of the table in a neat, human readable way.

In [30]:
# Open the connection using "with"

with engine.connect() as conn:
    
    # Loop through the tables with data in them
    
    for table in ["location", "weapon", "suspect"]:
        
        # Select id and name from each table
        
        results = conn.execute(f"SELECT id, name FROM {table};").fetchall()
        
        # Format the titles to stand out
        
        print("-" * 10)
        print(f"{table.title()}")
        print("-" * 10)
        
        # For each row (in each table)
        
        for row in results:
            
            # Print the id and name, separated by a colon
            
            print(": ".join([str(val) for val in row]))

----------
Location
----------
1: Kitchen
2: Ballroom
3: Conservatory
4: Dining Room
5: Cellar
6: Billiard Room
7: Library
8: Lounge
9: Hall
10: Study
----------
Weapon
----------
1: Candlestick
2: Dagger
3: Lead Pipe
4: Revolver
5: Rope
6: Wrench
----------
Suspect
----------
1: Black
2: Scarlett
3: Mustard
4: White
5: Green
6: Peacock
7: Plum


## Investigating the murder

With a working database, we can (finally) begin solving the mystery: who killed Mr. Black? The very stones of Tudor Close cry out for justice.

As Sherlock Holmes famously said, "Once you eliminate the impossible, whatever remains, no matter how improbable, must be the truth." He didn't say anything about SQL, but that is an understandable oversight, given when SQL was invented, and when his author died.

The next step in our investigation, having created the database, is to identify all the possible options. We can then start eliminating them one by one, leaving us with what must be - however improbable - the identity of the killer.

### Generating possibilities

We can add a row to the `accusation` table for each possible combination of suspect, location, and weapon. That's a lot of possibilities, and it would be tedious to do this by hand. However, we know that there are only a certain number of possible `id` values for the three things we care about; we can use Python to create a list of all possible combinations of those three possibilities.

This would be fiddly and annoying to code ourselves, but luckily there is a useful function in the `itertools` library that will do the awkward work for us, so that we can focus on serving Lady Justice.

In [31]:
# Create a list of all possible combinations

combinations = list(product([x for x in range(1,8)], # 7 possible suspect ids
                            [x for x in range(1, 11)],  # 10 possible locations  
                            [x for x in range(1, 7)]))  # 6 possible weapons

# View the first few combinations

combinations[:10]

[(1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 1, 4),
 (1, 1, 5),
 (1, 1, 6),
 (1, 2, 1),
 (1, 2, 2),
 (1, 2, 3),
 (1, 2, 4)]

Each `tuple` of numbers is  three ids - one for suspect, one for location, and one for weapon. This is exactly what is expected by the `accusation` table.

The next step is to add each of the possible combinations as a row in the `accusation` table. As we do not know which accusation is currently correct, we will refrain from marking any accusation as `True` or `False` (or, to be more precise, 1 or 0) at present.

The code to do this is very similar to the code we used to import the `weapon` data.

In [32]:
# Craft the statement

statement = """
INSERT INTO accusation
(suspect_id, location_id, weapon_id)
VALUES    
"""

# Loop through the combinations

for combination in combinations:

    # Add each possible guess to the statement
    
    statement = statement + f"({combination[0]}, {combination[1]}, {combination[2]}),\n"
    
# Remove the final character from the statement (trailing comma) and replace it with a semicolon

statement = statement[:-2] + ";"

# Display the first part of the statement (it's very long)

print(statement[:115])


INSERT INTO accusation
(suspect_id, location_id, weapon_id)
VALUES    
(1, 1, 1),
(1, 1, 2),
(1, 1, 3),
(1, 1, 4),


In [33]:
# Run the statement

with engine.connect() as conn:
    conn.execute(statement)

### Viewing possibilities

Once we have added the data to the database, we can view the various possibilities. Remember that the `accusation` table only stores the `id`s for the different parts of the accusation. We'll need to `JOIN` data together from different tables to get a human-readable list of possibilities.

Let's start by focusing on just one potential suspect: Miss Scarlett.

In [34]:
# Craft the query

query = """
    SELECT S.name AS suspect, L.name AS location, W.name AS weapon
    FROM accusation AS A
    LEFT JOIN suspect AS S
    ON (A.suspect_id = S.id)
    LEFT JOIN location AS L
    ON (A.location_id = L.id)
    LEFT JOIN weapon AS W
    ON (A.weapon_id = W.id)
    WHERE S.name = 'Scarlett';
"""

In [35]:
# Run the query

with engine.connect() as conn:
    scarlett_accusations = pd.read_sql(query, conn)

In [36]:
# View a sample of accusations

scarlett_accusations.sample(5)

,suspect,location,weapon
10,Scarlett,Ballroom,Rope
54,Scarlett,Study,Candlestick
31,Scarlett,Billiard Room,Dagger
16,Scarlett,Conservatory,Rope
40,Scarlett,Library,Rope


## Eliminating possibilities

Now that we have all of the possible accusations, we can start eliminating ones we know aren't true. Based on the original rules of Cluedo, we know the murder didn't take place in the cellar (`id` of 5), for example, and we know that (for it to be a murder) Dr. Black (`id` of 1) cannot be responsible.

We can mark all accusations involving one of the two above details as `False`, and remove them from the investigation.

In [37]:
# Craft the statement

statement = """
    UPDATE accusation
    SET true = 0
    WHERE location_id = 5
    OR suspect_id = 1;
"""

In [38]:
# Run the statement

with engine.connect() as conn:
    conn.execute(statement)

## Check remaining possibilities

We can write another query to count the number of accusations that aren't currently `False` (shown in the database by a `0`; if we're lucky, that number will be 1.

In [39]:
# Run the statement

with engine.connect() as conn:
    results = conn.execute("SELECT COUNT(*) FROM accusation WHERE true IS NOT 0").fetchall()
    print(results[0][0])

324


## Eliminating more possibilities

Unfortunately, that's still loads of possibilities. We're going to have to keep searching to find the murderer.

Luckily, we have some additional clues:

- The murder took place in a room with a one-word name
- The murder weapon's name is at least 6 letters long
- The murderer's name is shorter than the murder weapon's
- The weapon name does not start with the same letter as the location's
- The murder weapon's name ends with the same letter as the murder location's name starts.
- The length of the murderer's name, added to the murder location's name, is less than or equal to the murder weapon's name

Use these clues to cross off possibilities, marking accusations as `False` if they don't match the facts. You'll need to write and run SQL statements on the database, and will probably need to use the `LENGTH`, `LOWER`, and `SUBSTR` functions in SQLite.

You could, of course, work this all out without SQL, but where would be the fun in that?

Once you've eliminated the impossible, marking each inaccurate accusation with a `0` in the `true` column, then you should be able to use the cell below to reveal the real murderer.

In [45]:
statement = """
      SELECT A.id, S.honorific, S.name, L.name, W.name

      FROM accusation AS A
      LEFT JOIN suspect AS S
      ON (A.suspect_id = S.id)
      LEFT JOIN weapon AS W
      ON (A.weapon_id = W.id)
      LEFT JOIN location AS L
      ON (A.location_id = L.id)

      WHERE true IS NOT 0
"""

with engine.connect() as conn:
    
    results = conn.execute(statement).fetchall()
    
    if len(results) == 1:
        print(f"I accuse {results[0][1]} {results[0][2]}, in the {results[0][3]}, with the {results[0][4]}.")
    else:
        print(f"{len(results)} possibilities still under investigation.")

210 possibilities still under investigation.


<details>
    <summary>
        Reveal the culprit!
    </summary>
    I accuse Professor Plum, in the Kitchen, with the Candlestick.
</details